**Zadanie 1**  
Zainstaluj i uruchom Apache Airflow. Zaloguj się do panelu aplikacji www.

![Zadanie 1](zadanie_1.png)

**Zadanie 2**  
Uruchom zadanie `airflow/example_dags/tutorial.py` i prześledź w aplikacji jego wykonanie, log, wykres Gannt i inne statystyki, które są udostępniane przez interfejs. Na zakładce `Graph` po zaznaczeniu konkretnego zadania z grafu zmienia się ścieżka powrotu na górze okna i przechodząc teraz po zakładkach widzimy opis i inne właściwości zdefiniowane dla zadania, a nie całego grafu. Przeglądnij te informacje dla trzech zdefiniowanych zadań w tym grafie.

In [1]:
import textwrap
from datetime import datetime, timedelta

# The DAG object; we'll need this to instantiate a DAG
from airflow.models.dag import DAG

# Operators; we need this to operate!
from airflow.operators.bash import BashOperator

with DAG(
    "tutorial",
    # These args will get passed on to each operator
    # You can override them on a per-task basis during operator initialization
    default_args={
        "depends_on_past": False,
        "email": ["airflow@example.com"],
        "email_on_failure": False,
        "email_on_retry": False,
        "retries": 1,
        "retry_delay": timedelta(minutes=5),
        # 'queue': 'bash_queue',
        # 'pool': 'backfill',
        # 'priority_weight': 10,
        # 'end_date': datetime(2016, 1, 1),
        # 'wait_for_downstream': False,
        # 'sla': timedelta(hours=2),
        # 'execution_timeout': timedelta(seconds=300),
        # 'on_failure_callback': some_function, # or list of functions
        # 'on_success_callback': some_other_function, # or list of functions
        # 'on_retry_callback': another_function, # or list of functions
        # 'sla_miss_callback': yet_another_function, # or list of functions
        # 'on_skipped_callback': another_function, #or list of functions
        # 'trigger_rule': 'all_success'
    },
    description="A simple tutorial DAG",
    schedule=timedelta(days=1),
    start_date=datetime(2021, 1, 1),
    catchup=False,
    tags=["example"],
) as dag:

    # t1, t2 and t3 are examples of tasks created by instantiating operators
    t1 = BashOperator(
        task_id="print_date",
        bash_command="date",
    )

    t2 = BashOperator(
        task_id="sleep",
        depends_on_past=False,
        bash_command="sleep 5",
        retries=3,
    )
    t1.doc_md = textwrap.dedent(
        """\
    #### Task Documentation
    You can document your task using the attributes `doc_md` (markdown),
    `doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
    rendered in the UI's Task Instance Details page.
    ![img](https://imgs.xkcd.com/comics/fixing_problems.png)
    **Image Credit:** Randall Munroe, [XKCD](https://xkcd.com/license.html)
    """
    )

    dag.doc_md = __doc__  # providing that you have a docstring at the beginning of the DAG; OR
    dag.doc_md = """
    This is a documentation placed anywhere
    """  # otherwise, type it like this
    templated_command = textwrap.dedent(
        """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7)}}"
    {% endfor %}
    """
    )

    t3 = BashOperator(
        task_id="templated",
        depends_on_past=False,
        bash_command=templated_command,
    )

    t1 >> [t2, t3]

/opt/spark/work-dir/venv/lib/python3.10/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

![Zadanie 2](zadanie_2.png)

![Zadanie 2_2](zadanie_2_2.png)

![Zadanie 2_3](zadanie_2_3.png)

**Zadanie 3**  
Stwórz nowy graf poprzez modyfikację grafu z listingu 1 i zarejestruj go pod nazwą `tutorial_hourly` w pliku o nazwie `tutorial_hourly.py` i zapisz w tym samym folderze co oryginał. W definicji tego grafu zmodyfikuj:
* interwał uruchamiania z dziennego na godzinny,
* opis,
* dokumentację obiektu dag (widoczną później również poprzez interfejs graficzny),
* datę pierwszego uruchomienia na bieżącą,
* tag (np. lab8).

![Zadanie 3](zadanie_3.png)